# Page Rank and Recommendation Systems

### Imports

In [ ]:
!pip install scikit-network
from sknetwork.utils import get_neighbors; 
import sknetwork
from sknetwork.ranking import PageRank
from collections import defaultdict
import numpy as np; import pandas as pd

!pip install scikit-surprise scipy==1.4.1
from surprise import Dataset , Reader, prediction_algorithms
from surprise.model_selection import KFold, cross_validate, RandomizedSearchCV, GridSearchCV
import numpy as np; import pandas as pd 
import multiprocessing
from os import replace

## Page Rank
Using the provided dataset we will run various PageRank algorithms.
The dataset is an unweighted and undirected graph, where nodes represent characters from the "Harry Potter" books and an edge connects two characters in the graph if their names appeared at least one time within 14 words of one another in at least one of the books.

### Downloading the dataset 

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown 11Fa9zqDPgDthm3B84My1flWN6r-ffpY0 # download harry_potter_graph1.csv

df = pd.read_csv(r'/content/harry_potter_graph1.csv') # load

### Adjacency Matrix
Creating the adjacency matrix for the graph, assigning to each character a node identifier equal to the index that the character name has in ascending alphabetical order.

In [ ]:
records = df.to_records(index=False)
result = list(records)

Adj_matrix = sknetwork.data.from_edge_list(result,directed = False)

### Standard PageRank
Computing the PageRank vector for the obtained graph using a damping factor of 0.8.

In [ ]:
damping_factor = 0.8 # Set the parameters for the PageRank computation

# PageRank
pagerank = PageRank(damping_factor=damping_factor, solver="piteration", n_iter=1000, tol=10**-6)

# Compute Page-Rank on the entire WIKIPEDIA graph
pagerank_vector = pagerank.fit_transform(Adj_matrix)

Name and the PageRank score of the top-10 characters according to the PageRank score of their associated nodes.

In [ ]:
top10_characters = pd.Series(pagerank_vector).sort_values(ascending = False).index[:10]
result = zip(characters.iloc[top10_characters],pd.Series(pagerank_vector).sort_values(ascending = False))

for i in list(result):
    print(i)

('Harry Potter', 0.09148639794124665)
('Ronald Weasley', 0.0377128469364456)
('Rubeus Hagrid', 0.034322855633921345)
('Hermione Granger', 0.029204887471029867)
('Albus Dumbledore', 0.026936454399403835)
('Draco Malfoy', 0.022058455723732086)
('Neville Longbottom', 0.0215985445672487)
('Severus Snape', 0.02128201436448089)
('Dudley Dursley', 0.018598199943271058)
('Vernon Dursley', 0.015355015870565602)


### Topic-specific PageRank
Computing the Topic-specific PageRank vector for the obtained graph using a damping factor of 0.9, by considering as topic the Weasley family: a character belongs to the topic if its name contains the string `Weasley`.

In [ ]:
damping_factor = 0.9

# define the topic as probability distribution over all pages.
topic_as_map__page_id__landing_probability = {}
topic = set(np.where(characters.str.contains('Weasley'))[0])

pagerank = PageRank(damping_factor=damping_factor, solver="piteration", n_iter=1000, tol=10 ** -6)
num_pages_belonging_to_the_topic = len(topic)

for c_page_id_belonging_to_the_topic in topic:
    topic_as_map__page_id__landing_probability[c_page_id_belonging_to_the_topic] = 1. / num_pages_belonging_to_the_topic

# Compute Page-Rank on the entire WIKIPEDIA graph
pagerank_vector = pagerank.fit_transform(Adj_matrix, seeds=topic_as_map__page_id__landing_probability)

Name and PageRank score of the top-20 characters according to the Topic-specific PageRank score of their associated nodes.

In [ ]:
top20_topic_specific_characters = pd.Series(pagerank_vector).sort_values(ascending = False).index[:20]
result = zip(characters.iloc[top20_topic_specific_characters],pd.Series(pagerank_vector).sort_values(ascending = False))

for i in list(result): print(i)

('Harry Potter', 0.08638847973240897)
('Ronald Weasley', 0.07563217455276044)
('Fred Weasley', 0.0488022781561461)
('Percy Weasley', 0.04819506816368508)
('George Weasley', 0.044857815227002336)
('Rubeus Hagrid', 0.04166899845361425)
('Hermione Granger', 0.040928540047301405)
('Severus Snape', 0.03177222185978592)
('Albus Dumbledore', 0.030666633968766975)
('Neville Longbottom', 0.02855735065951652)
('Draco Malfoy', 0.02811852222369894)
('Argus Filch', 0.018016386207319476)
('Quirinus Quirrell', 0.017652339367384282)
('Filius Flitwick', 0.016014378892504025)
('Rolanda Hooch', 0.015328808733499977)
('Oliver Wood', 0.013814761365697727)
('Poppy Pomfrey', 0.013283981011776176)
('Peeves', 0.013125141816660331)
('Vernon Dursley', 0.01243625668149427)
('Seamus Finnigan', 0.012067612125960891)


### Personalized PageRank
Computing the Personalized PageRank vector for the obtained graph using a damping factor of 0.1 for each of the *Hogwarts professors*: Albus Dumbledore, Filius Flitwick, Firenze, Minerva McGonagall, Pomona Sprout, Quirinus Quirrell, Rolanda Hooch, Severus Snape.

In [ ]:
damping_factor = 0.1

prof_dict = defaultdict()
prof = {'Albus Dumbledore', 'Filius Flitwick', 'Firenze', 'Minerva McGonagall', 'Pomona Sprout', 'Quirinus Quirrell', 'Rolanda Hooch', 'Severus Snape'}

for p in prof:
 
  pagerank = PageRank(damping_factor=damping_factor, solver="piteration", n_iter=1000, tol=10 ** -6)
  topic = np.where(characters.str.contains(p))[0][0]
  
  # define the topic as probability distribution over all pages.
  map__page_id__landing_probability = {}
  map__page_id__landing_probability[topic] = 1.

  # Compute Page-Rank on the entire WIKIPEDIA graph
  prof_dict[p] = pd.Series(pagerank.fit_transform(Adj_matrix, seeds=map__page_id__landing_probability))

For each of the *Hogwarts professors*, Name and PageRank score of the top-3 characters according to the Personalized PageRank score of their associated nodes.

In [ ]:
for key in prof_dict:
  print('\n', key, ': \n')
  top3_personalized_characters = prof_dict[key].sort_values(ascending = False).index[:3]
  result = zip(characters.iloc[top3_personalized_characters], prof_dict[key].sort_values(ascending = False))

  for i in list(result):
      print(i)


 Rolanda Hooch : 

('Rolanda Hooch', 0.9007978604549658)
('Harry Potter', 0.009643772268233819)
('Marcus Flint', 0.009447364028106456)

 Filius Flitwick : 

('Filius Flitwick', 0.9012523255582493)
('Harry Potter', 0.007522154089004595)
('Rubeus Hagrid', 0.007390985801524833)

 Minerva McGonagall : 

('Minerva McGonagall', 0.9001087192041455)
('Harry Potter', 0.09023765774140742)
('Ronald Weasley', 0.00017638475484577383)

 Pomona Sprout : 

('Pomona Sprout', 0.9004920355342453)
('Harry Potter', 0.023081648236240523)
('Rubeus Hagrid', 0.023003309933920555)

 Firenze : 

('Firenze', 0.9007126647679351)
('Harry Potter', 0.01879735971184817)
('Rubeus Hagrid', 0.018728925094841017)

 Severus Snape : 

('Severus Snape', 0.900914492640715)
('Harry Potter', 0.0047108258088581385)
('Ronald Weasley', 0.004468580330940453)

 Albus Dumbledore : 

('Albus Dumbledore', 0.9016422923261983)
('Harry Potter', 0.004357769145351472)
('Rubeus Hagrid', 0.0040012249945617934)

 Quirinus Quirrell : 

('Quiri

### Topic-specific PageRank in an online context
Computing Topic-specific PageRank for the obtained graph using a damping factor of 0.1 considering an **online** context.

The Topic is the Hogwarts professors.

In [ ]:
aggregated_map__page_id__PersonalizedPageRank_score = {}
n = len(prof_dict)

for key in prof_dict.keys():

    for page_id, PersonalizedPageRank_score in prof_dict[key].items():

        if page_id not in aggregated_map__page_id__PersonalizedPageRank_score:
            aggregated_map__page_id__PersonalizedPageRank_score[page_id] = 0.

        aggregated_map__page_id__PersonalizedPageRank_score[page_id] += PersonalizedPageRank_score / n

aggregated_pr = list(aggregated_map__page_id__PersonalizedPageRank_score.items())

Name and the PageRank score of the top-10 characters according to the Topic-specific PageRank score of their associated nodes.

In [ ]:
idx = [] ; pr = []
for el in sorted(aggregated_pr, key=lambda tup: tup[1])[-10:][::-1]:
  idx.append(el[0]); pr.append(el[1])

result = zip(characters.iloc[idx], pr)

for i in list(result): print(i)

('Albus Dumbledore', 0.11642865404241108)
('Filius Flitwick', 0.11606050273075146)
('Severus Snape', 0.11493678629257519)
('Quirinus Quirrell', 0.11373421500975188)
('Pomona Sprout', 0.11348607396745644)
('Firenze', 0.11305802182590392)
('Rolanda Hooch', 0.11267541641280335)
('Minerva McGonagall', 0.11252487770806523)
('Harry Potter', 0.020648876132379666)
('Rubeus Hagrid', 0.008061659078349088)


## Recommendation Systems
In this part we'll improve the performance of various recommendation-systems by using non-trivial algorithms and also by performing the tuning of the hyper-parameters.

### Downloading the dataset 

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown 1MKE_PY0v30CP_rs3sNzwkmo6QvKAVn_J 

file_path = '/content/ratings_hw2.csv'
reader = Reader(line_format='user item rating', sep=',', rating_scale=[3, 5], skip_lines=1)

data = Dataset.load_from_file(file_path, reader=reader) # surprise data structure

### Recommendation algorithms
Defining **all** the algorithms we are going to use

In [ ]:
#YOUR CODE STARTS HERE#
NP = prediction_algorithms.random_pred.NormalPredictor()
BO = prediction_algorithms.baseline_only.BaselineOnly()
KNNB = prediction_algorithms.knns.KNNBasic()
KNNM = prediction_algorithms.knns.KNNWithMeans()
KNNZ = prediction_algorithms.knns.KNNWithZScore()
KNNBaseline = prediction_algorithms.knns.KNNBaseline()
SVD = prediction_algorithms.matrix_factorization.SVD()
SVD_pp = prediction_algorithms.matrix_factorization.SVDpp()
NMF = prediction_algorithms.matrix_factorization.NMF()
SlopeOne = prediction_algorithms.slope_one.SlopeOne()
CoClust = prediction_algorithms.co_clustering.CoClustering()

### Parameter configuration
Defining the parameter configurations for each selected algorithm.

In [ ]:

baseline_predictor_options = {   
  'method': "sgd",  # Optimization method to use.
  'learning_rate': 0.005,  # Learning rate parameter for the SGD optimization method.
  'n_epochs': 50,  # The number of iteration for the SGD optimization method.
  'reg': 0.02,  # The regularization parameter of the cost function that is optimized: a.k.a. LAMBDA.
}

BO = prediction_algorithms.baseline_only.BaselineOnly(bsl_options=baseline_predictor_options, verbose=True)

KNNB = prediction_algorithms.knns.KNNBasic(k=40, min_k=1, verbose=True)

KNNM = prediction_algorithms.knns.KNNWithMeans(k=40, min_k=1, verbose=True)

KNNZ = prediction_algorithms.knns.KNNWithZScore(k=40, min_k=1, verbose=True)

KNNBaseline = prediction_algorithms.knns.KNNBaseline(k=40, min_k=1, verbose=True)

SVD = prediction_algorithms.matrix_factorization.SVD(n_factors=100,biased=True,n_epochs=20,lr_all=0.005,reg_all=0.02,verbose=True)

SVD_pp = prediction_algorithms.matrix_factorization.SVDpp(n_factors=20,n_epochs=20,lr_all=0.007,reg_all=0.02,verbose=True)

NMF = prediction_algorithms.matrix_factorization.NMF()

SlopeOne = prediction_algorithms.slope_one.SlopeOne()

CoClust = prediction_algorithms.co_clustering.CoClustering()

alg=[NP,BO,KNNB,KNNM,KNNZ,KNNBaseline,SVD,SVD_pp,NMF,SlopeOne,CoClust]

### K-fold cross validation

Initializing a `scikit-surprise` `KFold` object with 5-folds. Random_state set to **42**.


Running a **5-Folds**-cross-validation for all the selected algorithms, using the parameters configuration selected.

In [ ]:
kf = KFold(n_splits=5, random_state=42)

mean_score=[]
for method in alg:
    cv = cross_validate(method, data, measures=['MAE'], cv=kf, verbose=True,n_jobs= -1)
    print('\n')
    mean_score.append((sum(cv['test_mae'])/5))

Evaluating MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.6107  0.6151  0.6106  0.6146  0.6142  0.6131  0.0020  
Fit time          0.12    0.12    0.12    0.12    0.06    0.11    0.02    
Test time         0.25    0.25    0.24    0.25    0.14    0.23    0.04    


Evaluating MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.3732  0.3736  0.3693  0.3698  0.3722  0.3716  0.0018  
Fit time          0.70    0.75    1.12    0.71    0.37    0.73    0.24    
Test time         0.17    0.51    0.18    0.16    0.10    0.22    0.15    


Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.3898  0.3921  0.3845  0.3872  0.3888  0.3885  0.0025  
Fit time          0.43    0.61    0.66    0.64    0.44    0.56    0.10    
T

### Rank
Ranking all recommendation algorithms according to the mean of the Mean Absolute Error metric value: from the best to the worst algorithm.

In [ ]:
names = ("NormalPredictor","BaselineOnly","KNNBasic","KNNWithMeans","KNNWithZScore","KNNBaseline","SVD","SVD++","NMF","SlopeOne","CoClust")

result = list(zip(names, mean_score))

data = pd.DataFrame(result)

data.sort_values(1, ascending=True)

,0,1
5,KNNBaseline,0.367445
7,SVD++,0.370623
1,BaselineOnly,0.371619
9,SlopeOne,0.373288
4,KNNWithZScore,0.374053
3,KNNWithMeans,0.375658
6,SVD,0.375860
2,KNNBasic,0.388473
8,NMF,0.406271
10,CoClust,0.440851
